In [0]:
# No validation set.

# Environment Setting

In [0]:
! pip3 install --upgrade tensorflow-gpu
! pip3 install --upgrade keras

     |████████████████████████████████| 380.8MB 42kB/s 
     |████████████████████████████████| 3.8MB 38.2MB/s 
     |████████████████████████████████| 450kB 68.4MB/s 
     |████████████████████████████████| 81kB 13.4MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found exis

     |████████████████████████████████| 378kB 4.8MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


### Test if tf can see GPU

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(f'Fonnd GPU device at {device_name}')

Fonnd GPU device at /device:GPU:0


# Datasets Preparation

In [0]:
# Upload data(data.zip)----Based on: https://www.youtube.com/watch?v=0rygVrmHidg&list=LLADe5D3g-jb62XZ74pwv9EA&index=3&t=170s

from zipfile import ZipFile

file_name = "data.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [0]:
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout, GlobalAveragePooling2D, BatchNormalization

# import warnings
# warnings.filterwarnings('ignore')

In [0]:
# path = '/content/drive/My Drive/Colab Notebooks/MRG Project/data'

# train_path = f'{path}/train'
# test_path = f'{path}/test'

train_path = 'data/train'
test_path = 'data/test'

In [0]:
# train: test = 8: 2
# train = 800, test = 200

img_size = (224,224)
input_shape =(224,224,3)
num_classes = 10


train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=img_size, 
                                                         class_mode='sparse', batch_size=50)

test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=img_size, 
                                                        class_mode='sparse', batch_size=10)

Found 734 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


# Train & Predict

## Model1 (Jump to Model2/VGG...)
Based on: https://www.youtube.com/watch?v=LhEMXbjGV_4

In [0]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

In [0]:
model.fit_generator(train_batches, steps_per_epoch=12, epochs=20)    # steps_per_epoch = 600/50 = 12

In [0]:
model.evaluate_generator(test_batches, steps=20)

[25.360055923461914, 0.33000001311302185]

## Model 2
Based on: https://medium.com/x8-the-ai-community/audio-classification-using-cnn-coding-example-f9cbd272269e

In [0]:
model = Sequential()

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

In [0]:
model.fit_generator(train_batches, steps_per_epoch=12, epochs=100)

In [0]:
model.evaluate_generator(test_batches, steps=20)

[2.306565761566162, 0.10000000149011612]

## Model3 -- VGG16
Based on: https://www.youtube.com/watch?v=oDHpqu52soI

### 1.Adam(lr=.0001)

In [0]:
from keras.applications.vgg16 import VGG16

vgg16_model = VGG16()
vgg16_model.layers.pop()

from keras.layers import InputLayer

new_model = Sequential()
new_model.add(InputLayer(input_shape=input_shape))

for layer in vgg16_model.layers:
  new_model.add(layer)

for layer in new_model.layers:
  layer.trainable=False

new_model.add(Dense(num_classes, activation='softmax'))

new_model.summary()

from keras.optimizers import Adam

new_model.compile(optimizer=Adam(lr=.0001),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [0]:
new_model.fit_generator(train_batches, steps_per_epoch=12, epochs=100)

Epoch 1/100
12/12 [==============================] - 3s 243ms/step - loss: 2.4954 - accuracy: 0.1421
Epoch 2/100
12/12 [==============================] - 3s 230ms/step - loss: 2.1530 - accuracy: 0.2003
Epoch 3/100
12/12 [==============================] - 3s 233ms/step - loss: 2.0013 - accuracy: 0.2774
Epoch 4/100
12/12 [==============================] - 3s 239ms/step - loss: 1.8072 - accuracy: 0.3667
Epoch 5/100
12/12 [==============================] - 3s 228ms/step - loss: 1.7119 - accuracy: 0.4144
Epoch 6/100
12/12 [==============================] - 3s 228ms/step - loss: 1.6151 - accuracy: 0.4658
Epoch 7/100
12/12 [==============================] - 3s 241ms/step - loss: 1.5261 - accuracy: 0.4883
Epoch 8/100
12/12 [==============================] - 3s 223ms/step - loss: 1.4667 - accuracy: 0.5335
Epoch 9/100
12/12 [==============================] - 3s 237ms/step - loss: 1.3609 - accuracy: 0.5733
Epoch 10/100
12/12 [==============================] - 3s 231ms/step - loss: 1.3635 - accura

In [0]:
new_model.evaluate_generator(test_batches, steps=20)

[1.245793104171753, 0.6700000166893005]

### 2.'adam' 

In [0]:
from keras.applications.vgg16 import VGG16

vgg16_model = VGG16()
vgg16_model.layers.pop()

from keras.layers import InputLayer

new_model = Sequential()
new_model.add(InputLayer(input_shape=input_shape))

for layer in vgg16_model.layers:
  new_model.add(layer)

for layer in new_model.layers:
  layer.trainable=False

new_model.add(Dense(num_classes, activation='softmax'))

new_model.summary()

new_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [0]:
new_model.fit_generator(train_batches, steps_per_epoch=12, epochs=10)

In [0]:
new_model.evaluate_generator(test_batches, steps=20)

[1.4042872190475464, 0.620512843132019]

In [0]:
# # How to save a model

# new_model.save('/content/drive/My Drive/Colab Notebooks/MRG Project/models/m1.h5')

# from keras.models import load_model

# model = load_model('/content/drive/My Drive/Colab Notebooks/MRG Project/models/m1.h5')

# model.evaluate_generator(test_batches, steps=20)

### 3.SGD (as vgg16)
Bad results!

In [0]:
from keras.applications.vgg16 import VGG16

vgg16_model = VGG16()
vgg16_model.layers.pop()

from keras.layers import InputLayer

new_model = Sequential()
new_model.add(InputLayer(input_shape=input_shape))

for layer in vgg16_model.layers:
  new_model.add(layer)

for layer in new_model.layers:
  layer.trainable=False

new_model.add(Dense(num_classes, activation='softmax'))

new_model.summary()

from keras.optimizers import SGD

new_model.compile(optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [0]:
new_model.fit_generator(train_batches, steps_per_epoch=12, epochs=80)

# New Model 1
https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7

In [0]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [0]:
model.fit_generator(train_batches, steps_per_epoch=12, epochs=50)

In [0]:
model.evaluate_generator(test_batches, steps=20)

[1.2128499746322632, 0.3499999940395355]

# New Model 2
https://github.com/derekahuang/Music-Classification/blob/master/cnn.py

In [0]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(4, 4),
                activation='relu', #kernel_regularizer=regularizers.l2(0.04),
                input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 4)))
model.add(Conv2D(64, (3, 5), activation='relu'
                # , kernel_regularizer=regularizers.l2(0.04)
                ))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (2, 2), activation='relu'
  # , kernel_regularizer=regularizers.l2(0.04)
    ))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu', ))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

In [0]:
new_model.fit_generator(train_batches, steps_per_epoch=12, epochs=20)

Epoch 1/20
12/12 [==============================] - 3s 248ms/step - loss: 0.4007 - accuracy: 0.9267
Epoch 2/20
12/12 [==============================] - 3s 237ms/step - loss: 0.3824 - accuracy: 0.9229
Epoch 3/20
12/12 [==============================] - 3s 237ms/step - loss: 0.3649 - accuracy: 0.9419
Epoch 4/20
12/12 [==============================] - 3s 239ms/step - loss: 0.4073 - accuracy: 0.9200
Epoch 5/20
12/12 [==============================] - 3s 235ms/step - loss: 0.3527 - accuracy: 0.9418
Epoch 6/20
12/12 [==============================] - 3s 238ms/step - loss: 0.3734 - accuracy: 0.9384
Epoch 7/20
12/12 [==============================] - 3s 240ms/step - loss: 0.3657 - accuracy: 0.9298
Epoch 8/20
12/12 [==============================] - 3s 238ms/step - loss: 0.3756 - accuracy: 0.9281
Epoch 9/20
12/12 [==============================] - 3s 245ms/step - loss: 0.3527 - accuracy: 0.9417
Epoch 10/20
12/12 [==============================] - 3s 234ms/step - loss: 0.3569 - accuracy: 0.9349

In [0]:
new_model.evaluate_generator(test_batches, steps=20)

[1.0326225757598877, 0.6899999976158142]